# Scrape data of stay-at-home orders

Data is scraped from

https://www.kff.org/coronavirus-policy-watch/stay-at-home-orders-to-fight-covid19/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pymongo import MongoClient
import pprint

import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

### Step 1: Check out the website in a browser.

In [3]:
stay_at_home_url = 'https://www.kff.org/coronavirus-policy-watch/stay-at-home-orders-to-fight-covid19/'

### Step 2: Send a GET request for the data.

In [4]:
r = requests.get(stay_at_home_url)

In [5]:
r.status_code

200

### Step 3: Save all the hypertext into mongo for later use.

In [7]:
client = MongoClient('localhost', 27017)
db = client.kff_org
pages = db.pages

pages.insert_one({'html': r.content})

### Step 4: Parse the hypertext with BeautifulSoup

In [8]:
soup = BeautifulSoup(r.content, "html")

In [42]:
#print(soup.prettify())

### Step 5: Navigate the data to pull out the table information.

In [57]:
table = soup.find("table", {"class": 
                            "less-padding alternate-highlights-gray"})
rows = table.find_all("tr")

rows[3].text.split('\n')

#print(type(table))
#print(type(rows))

['', 'Alaska', 'March 27', 'March 28', '']

In [62]:
all_rows = []

# Store each row as a dictionary

empty_row = {
    "state": None, "Date Announced": None, "Effective Date": None
}

# The first two rows contains header information, skip it. 
# The last row contains footer information, skip that too.

for row in rows[2:len(rows)-1]:
    new_row = copy.copy(empty_row)
    each_row = row.text.split('\n')
    new_row['state'] = each_row[1]
    new_row['Date Announced'] = each_row[2]
    new_row['Effective Date'] = each_row[3]

    all_rows.append(new_row)

In [64]:
#all_rows

### Step 6: Load all the rows into a Mongo database.

In [65]:
db = client.kff_org
stay_home_states = db.stay_home_states

In [66]:
for row in all_rows:
    stay_home_states.insert_one(row)

### Step 7: Load all the rows into a pandas dataframe

In [68]:
stay_home_orders_state = pd.DataFrame(all_rows)

### Step 8: Save pandas dataframe to a CSV.

In [70]:
stay_home_orders_state.to_csv('datasets/stay_home_orders_state.csv')

### Step 9: Convert dates to datetimes

#### Step 9a: Read in CSV.

In [3]:
import pandas as pd

In [11]:
from datetime import datetime

In [16]:
import numpy as np

In [81]:
stay_home_orders_state = pd.read_csv('../datasets/stay_home_orders_state.csv', index_col = 'state')

In [82]:
stay_home_orders_state.head()

,Unnamed: 0,Date Announced,Effective Date,_id
state,,,,
Alabama,0,April 3,April 4,5e8b9794aded4fdccab0eca4
Alaska,1,March 27,March 28,5e8b9794aded4fdccab0eca5
Arizona,2,March 30,March 31,5e8b9794aded4fdccab0eca6
Arkansas,3,–,–,5e8b9794aded4fdccab0eca7
California,4,March 19,March 19,5e8b9794aded4fdccab0eca8


In [83]:
stay_home_orders_state.drop('Unnamed: 0', axis=1, inplace=True)

In [84]:
stay_home_orders_state.head()

,Date Announced,Effective Date,_id
state,,,
Alabama,April 3,April 4,5e8b9794aded4fdccab0eca4
Alaska,March 27,March 28,5e8b9794aded4fdccab0eca5
Arizona,March 30,March 31,5e8b9794aded4fdccab0eca6
Arkansas,–,–,5e8b9794aded4fdccab0eca7
California,March 19,March 19,5e8b9794aded4fdccab0eca8


#### 9b: Add year (2020) to dates

In [85]:
stay_home_orders_state['Date Announced'] = stay_home_orders_state['Date Announced'] + ' 2020'

In [87]:
stay_home_orders_state['Effective Date'] = stay_home_orders_state['Effective Date'] + ' 2020'

In [88]:
stay_home_orders_state.head()

,Date Announced,Effective Date,_id
state,,,
Alabama,April 3 2020,April 4 2020,5e8b9794aded4fdccab0eca4
Alaska,March 27 2020,March 28 2020,5e8b9794aded4fdccab0eca5
Arizona,March 30 2020,March 31 2020,5e8b9794aded4fdccab0eca6
Arkansas,– 2020,– 2020,5e8b9794aded4fdccab0eca7
California,March 19 2020,March 19 2020,5e8b9794aded4fdccab0eca8


In [90]:
a = stay_home_orders_state.loc['Arkansas', 'Date Announced']
a

'– 2020'

In [91]:
stay_home_orders_state.replace({'Date Announced':a, 'Effective Date':a},'-', inplace=True)

In [92]:
stay_home_orders_state.head()

,Date Announced,Effective Date,_id
state,,,
Alabama,April 3 2020,April 4 2020,5e8b9794aded4fdccab0eca4
Alaska,March 27 2020,March 28 2020,5e8b9794aded4fdccab0eca5
Arizona,March 30 2020,March 31 2020,5e8b9794aded4fdccab0eca6
Arkansas,-,-,5e8b9794aded4fdccab0eca7
California,March 19 2020,March 19 2020,5e8b9794aded4fdccab0eca8


#### 9c: Drop states with no dates

In [106]:
stay_home_orders_state = stay_home_orders_state[stay_home_orders_state['Date Announced'] != '-'].copy()

In [107]:
stay_home_orders_state.head()

,Date Announced,Effective Date,_id
state,,,
Alabama,2020-04-03,April 4 2020,5e8b9794aded4fdccab0eca4
Alaska,2020-03-27,March 28 2020,5e8b9794aded4fdccab0eca5
Arizona,2020-03-30,March 31 2020,5e8b9794aded4fdccab0eca6
California,2020-03-19,March 19 2020,5e8b9794aded4fdccab0eca8
Colorado,2020-03-26,March 26 2020,5e8b9794aded4fdccab0eca9


#### 9d: Convert dates to datetimes

In [111]:
stay_home_orders_state['Effective Date'] = (stay_home_orders_state['Effective Date'].apply(lambda x: datetime.strptime(x, '%B %d %Y')))
stay_home_orders_state['Date Announced'] = (stay_home_orders_state['Effective Date'].apply(lambda x: datetime.strptime(x, '%B %d %Y')))

In [112]:
stay_home_orders_state.head()

,Date Announced,Effective Date,_id
state,,,
Alabama,2020-04-03,2020-04-04,5e8b9794aded4fdccab0eca4
Alaska,2020-03-27,2020-03-28,5e8b9794aded4fdccab0eca5
Arizona,2020-03-30,2020-03-31,5e8b9794aded4fdccab0eca6
California,2020-03-19,2020-03-19,5e8b9794aded4fdccab0eca8
Colorado,2020-03-26,2020-03-26,5e8b9794aded4fdccab0eca9


#### 9e: Drop the _id column

In [115]:
stay_home_orders_state.drop('_id', axis=1, inplace=True)

In [116]:
stay_home_orders_state.head()

,Date Announced,Effective Date
state,,
Alabama,2020-04-03,2020-04-04
Alaska,2020-03-27,2020-03-28
Arizona,2020-03-30,2020-03-31
California,2020-03-19,2020-03-19
Colorado,2020-03-26,2020-03-26


#### 9f: Save this format into a CSV

In [118]:
stay_home_orders_state.to_csv('../datasets/stay_home_orders_state_with_dates.csv')

## Step 10: Pickle the data

In [123]:
stay_home_orders_state.to_pickle('../datasets/stay_home_orders_pickled.pkl')

In [124]:
b = [1, 2, 3, 4, 5, 6]

[1, 2, 3, 4, 5]

In [125]:
unpickled = pd.read_pickle('../datasets/stay_home_orders_pickled.pkl')

In [126]:
unpickled.head()

,Date Announced,Effective Date
state,,
Alabama,2020-04-03,2020-04-04
Alaska,2020-03-27,2020-03-28
Arizona,2020-03-30,2020-03-31
California,2020-03-19,2020-03-19
Colorado,2020-03-26,2020-03-26


In [127]:
type(unpickled.iloc[1,1])

pandas._libs.tslibs.timestamps.Timestamp